In [1]:
import sys
sys.path.append("../")
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from utils.inkml2img import convert_dir
from tqdm.auto import tqdm
from data.dataset import Im2LatexDataset
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from model.vit import ViT
import lightning as L
import wandb


/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
img_dims = [224, 640]
model = ViT(img_dims, 16, n_embd=512, encoder=False, output_classes=10)
data = Im2LatexDataset(path_to_data="../data/",
                       tokenizer="../data/tokenizer.json", img_dims=img_dims, classification=True, batch_size=16, device=torch.device('mps'))
batch = next(iter(data.train))

In [3]:
model = ViT(img_dims, 16, n_embd=512, encoder=False, output_classes=1)
model.training_step(batch, 1)

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/core/module.py:407: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


tensor(0.5619, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [4]:
class ImagePredictionLogger(L.Callback):
    def __init__(self, val_samples, num_samples=10):
        super().__init__()
        self.val_imgs, self.val_labels = val_samples
        self.val_imgs = self.val_imgs[:num_samples]
        self.val_labels = self.val_labels[:num_samples]

    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)

        logits = pl_module(val_imgs)
        preds = F.sigmoid(logits)

        trainer.logger.experiment.log({
            "examples": [wandb.Image(x, caption=f"Pred:{pred}, Label:{y}")
                         for x, pred, y in zip(val_imgs, preds, self.val_labels)],
            "global_step": trainer.global_step
        })


# checkpoint_callback = ModelCheckpoint(monitor='train accuracy', mode='max')
logger = WandbLogger(project='vittest')

trainer = L.Trainer(limit_train_batches=16, max_epochs=5, log_every_n_steps=16, deterministic=True,
                    logger=logger, callbacks=[ImagePredictionLogger(batch)], accelerator='mps')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: baolong. Use `wandb login --relogin` to force relogin


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
trainer.fit(model, data.train, data.val)
wandb.finish()


  | Name               | Type            | Params
-------------------------------------------------------
0 | patch_embeddings   | PatchEmbeddings | 131 K 
1 | pos_dropout        | Dropout         | 0     
2 | transformer_blocks | ModuleList      | 18.9 M
3 | ln                 | LayerNorm       | 1.0 K 
4 | accuracy           | BinaryAccuracy  | 0     
5 | head               | Linear          | 513   
-------------------------------------------------------
19.3 M    Trainable params
0         Non-trainable params
19.3 M    Total params
77.279    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/16 [00:00<?, ?it/s] 